In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchains);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = a, bp, a_society.1, a_society.2, a_society.3, a_society.4, a_society.5, a_society.6, a_society.7, a_society.8, a_society.9, a_society.10, sigma_society

Empirical Posterior Estimates
──────────────────────────────────────────────────────
parameters
                Mean    SD   Naive SE  MCSE     ESS   
            a  1.1210 0.7399   0.0117 0.0204 1000.0000
  a_society.1 -0.2049 0.2386   0.0038 0.0050 1000.0000
  a_society.2  0.0418 0.2148   0.0034 0.0047 1000.0000
  a_society.3 -0.0399 0.1865   0.0029 0.0037 1000.0000
  a_society.4  0.3278 0.1919   0.0030 0.0056 1000.0000
  a_society.5  0.0437 0.1751   0.0028 0.0037 1000.0000
  a_society.6 -0.3172 0.2004   0.0032 0.0044 1000.0000
  a_society.7  0.1510 0.1710   0.0027 0.0043 1000.0000
  a_society.8 -0.1663 0.1810   0.0029 0.0042 1000.0000
  a_society.9  0.2789 0.1775   0.0028 0.0044 1000.00

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*